In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Dense,Attention,GlobalMaxPooling1D,GlobalAveragePooling1D
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from keras.preprocessing.text import Tokenizer,text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.layers import AveragePooling1D,Bidirectional, Conv1D,MaxPooling1D,Embedding, LSTM, Input, Flatten, Dense, Dropout, Convolution2D, Conv2D, MaxPooling2D, Lambda, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, AveragePooling2D, Concatenate
from sklearn.model_selection import train_test_split, StratifiedKFold
from keras.utils.np_utils import to_categorical
from keras.layers import Flatten,Layer
from sklearn.model_selection import KFold
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers,activations,initializers,regularizers,constraints
import re

In [2]:
# load the dataset
df= pd.read_csv('/Users/yuwenyu/Desktop/PhD./fake_news/Constraint_English_Train.csv')
df_val= pd.read_csv('/Users/yuwenyu/Desktop/PhD./fake_news/Constraint_English_Val.csv')
df_test= pd.read_csv('/Users/yuwenyu/Desktop/PhD./fake_news/Constraint_English_Test.csv')

#df= pd.read_csv('Constraint_English_Train.csv')
#df_val= pd.read_csv('fake_news/Constraint_English_Val.csv')
#df_test= pd.read_csv('Constraint_English_Test.csv')

In [3]:
df_all = pd.concat([df, df_val])
X_train=df_all['tweet']
y_train=df_all['label']
X_test= df_test['tweet']
y_test=df_test['label']


In [6]:
# encoding label
encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test= encoder.fit_transform(y_test)


In [7]:
# Tokenizer
tokenizer = Tokenizer()

tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
tokenizer.fit_on_texts(X_test)
X_test = tokenizer.texts_to_sequences(X_test)

In [8]:
# Pad_sequence
from keras.preprocessing.sequence import pad_sequences
input_length=256 #1420
X_train = pad_sequences(X_train, padding='post', maxlen=input_length)
X_test = pad_sequences(X_test, padding='post', maxlen=input_length)


In [9]:
#word embedding
emb_dim=300
vocab=len(tokenizer.word_index)+1
#print(vocab)

emb_mat= np.zeros((vocab,emb_dim))

with open('/Users/yuwenyu/Desktop/PhD./fake_news/glove.6B.300d.txt') as f:
    for line in f:
        word,*emb = line.split()
        if word in tokenizer.word_index:
            ind=tokenizer.word_index[word]
            emb_mat[ind]=np.array(emb,dtype="float32")[:emb_dim]


27418


In [ ]:
# Model
model = Sequential()
model.add(Embedding(vocab,300,weights=[emb_mat], input_length=input_length,trainable=False))
model.add(Bidirectional(LSTM(300, return_sequences=True)))
model.add(Dropout(0.4))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=100))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

history=model.fit(X_train,y_train,epochs=10,batch_size=256, verbose=2,shuffle=True)


In [ ]:
# Final evaluation of the model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_pred=model.predict(X_test)
print('Classification Report:')
print(classification_report(y_test, y_pred, labels=[1,0], digits=4))
